In [2]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [44]:
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup #all data collection i got help from https://www.youtube.com/watch?v=yME299lFvFk
import requests
import nltk
import string
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
stemmer = nltk.stem.PorterStemmer()
stopwords = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer()
vocab = []
vocab_counts = []
vocab_dic = 0

In [5]:
Agriculture_urls = [
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg52370/html/CHRG-118hhrg52370.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg52205/html/CHRG-118hhrg52205.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg52557/html/CHRG-118hhrg52557.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg52371/html/CHRG-118hhrg52371.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg54212/html/CHRG-118hhrg54212.htm"
]

Armed_services_urls = [
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg51700/html/CHRG-118hhrg51700.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg51704/html/CHRG-118hhrg51704.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg51705/html/CHRG-118hhrg51705.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg53350/html/CHRG-118hhrg53350.htm",
    "https://www.govinfo.gov/content/pkg/CHRG-118hhrg52380/html/CHRG-118hhrg52380.htm"
]


My dataset consists of each paragraph in each of the hearings as documents.

In [6]:
#this method I got help from chatgpt to pull the txt from the wepages
def url_to_txt (url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.find('body')
  text = body.get_text(separator='\n')
  return text

def text_splitter(doc):
  dump = []
  doc = str(doc)
  docs = doc.split("    ")
  for doc_len in docs:
    if len(doc_len) >= 150:
      dump.append(doc_len)
  print(len(dump))
  return dump

#https://spotintelligence.com/2022/12/21/nltk-preprocessing-pipeline/
def preprocessing_doc(docs):
  dump = []
  for i in range(len(docs)):
    for j in range(len(docs[i])):
      tokens = nltk.word_tokenize(str(docs[i][j]))
      tokens = [token.lower() for token in tokens]
      tokens = [token for token in tokens if token not in string.punctuation]
      tokens = [token for token in tokens if token.lower() not in stopwords]
      tokens = [token for token in tokens if not any(char.isdigit() for char in token)]#this line i got help for checking the entire stirng for any digits
      stemmed_tokens = [stemmer.stem(token) for token in tokens]
      docs[i][j] = stemmed_tokens
      dump.append(docs[i][j])
  return dump




def pad_sequence(sequence, max_length):
    return sequence + [0] * (max_length - len(sequence))

def bag_of_words(docs_1, docs_2):
  total_len = len(docs_1) + len(docs_2)
  max_length = max(len(doc) for doc in docs_1 + docs_2)

  print(total_len, len(docs_1), len(docs_2))
  padded_docs_1 = [pad_sequence(doc, max_length) for doc in docs_1]
  padded_docs_2 = [pad_sequence(doc, max_length) for doc in docs_2]


  #I was haveing issues with the input size for using the np libray this following line and padded_seq funciton i got help from chatgpt to fix bug with not same dimensions for vocab def
  docs_np = np.concatenate((padded_docs_1, padded_docs_2))

  vocab, vocab_counts = np.unique(docs_np.flatten(), return_counts=True)
  vocab_dic = {vocab[x]: x for x in range(len(vocab))}
  bag_of_words = np.zeros((total_len, len(vocab)))


  for i in tqdm(range(len(padded_docs_1))):
      for token in padded_docs_1[i]:
        if token != 0:
          bag_of_words[i][vocab_dic[token]] += 1

  for i in tqdm(range(len(padded_docs_2))):
      for token in padded_docs_2[i]:
        if token != 0:
          bag_of_words[i + len(padded_docs_1)][vocab_dic[token]] += 1
  print(bag_of_words.shape)
  return bag_of_words, vocab, vocab_counts,vocab_dic

def naive_bayes(bag_of_words, vocab, vector='count', count_min=15):
    # Filter out words with counts below the threshold
    docs_1 = bag_of_words[:1712]
    docs_2 = bag_of_words[1712:]

    counts_docs_1 = np.sum(docs_1, axis=0)
    counts_docs_2 = np.sum(docs_2, axis=0)

    # Filter out words with counts below count_min
    index_doc_1 = np.where(counts_docs_1 > count_min)[0]
    index_doc_2 = np.where(counts_docs_2 > count_min)[0]

    # Compute probabilities using filtered counts
    prob_1 = np.log(np.divide(counts_docs_1[index_doc_1], counts_docs_2[index_doc_1] + 0.001))
    prob_2 = np.log(np.divide(counts_docs_2[index_doc_2], counts_docs_1[index_doc_2] + 0.001))

    # Select top 50 words
    doc_1_10 = np.argsort(prob_1)[-50:]
    doc_2_10 = np.argsort(prob_2)[-50:]

    docs_1_10_w = vocab[doc_1_10]
    docs_2_10_w = vocab[doc_2_10]

    return docs_1_10_w, docs_2_10_w


In [7]:
agr_docs = np.array([url_to_txt(doc) for doc in Agriculture_urls])
as_docs = np.array([url_to_txt(doc) for doc in Armed_services_urls])


In [8]:
new_agr_docs = []
for doc in agr_docs:
    new_agr_docs.append(text_splitter(doc))
agr_docs = new_agr_docs

# Process documents in as_docs
new_as_docs = []
for doc in as_docs:
    new_as_docs.append(text_splitter(doc))
as_docs = new_as_docs

1591
352
560
243
866
64
503
658
156
331


In [9]:
agr_docs = preprocessing_doc(agr_docs)
as_docs = preprocessing_doc(as_docs)

In [10]:
bow, vocab,vocab_counts,vocab_dic =  bag_of_words(as_docs,agr_docs)

5324 1712 3612


100%|██████████| 3612/3612 [00:00<00:00, 6004.05it/s]


(5324, 9210)


In [11]:
w_1, w_2 = naive_bayes(bow, vocab)
print(w_1, w_2)

['afford' 'aia' 'all-in' 'amount'
 '.................................................' 'advantag'
 'all-of-the-abov' 'almost' 'anchor' 'arrest'
 '//www.ams.usda.gov/reports/agricultural-competition-' 'asylum'
 'acquisit' 'airborn' 'ambul' 'apa' '//www.ers.usda.gov/data-' 'amplif'
 '//www.fsis.usda.gov/shared/pdf/' 'action-ori' 'ag-'
 '//www.fsis.usda.gov/news-' 'amort' 'approach' 'alon' 'apart' 'acquir'
 '....................................................' 'alli' 'attorney'
 'almond' 'agra' '//www.nationalchickencouncil.org/' 'along' 'ams-' 'acid'
 'among' 'arab' 'altitud' 'apprentic' 'allud' 'acep'
 '//www.ers.usda.gov/topics/farm-economy/farm-sector-' 'alter'
 'apprenticeship' 'astorga' 'aquat' 'across' 'age/sex'
 '.......................................'] ['basket' 'aviagen' 'chinese-own' 'asylum' '//democrats-' 'backyard'
 'andrew' 'ala.' 'cline' 'civil' 'aris' 'autom' 'badli' 'brown' 'barri'
 'agrosecur' 'brave' 'alpha'
 '//www.aei.org/research-products/report/subsidized-beef-' 

# 2.4 Topic Modeling


---


Run Latent Dirichlet Allocation using all of the documents in your corpus. You
may choose the number of topics that you feel is most appropriate and gives
the results that either look most reasonable to you or optimize a metric like
coherence. You do not need to implement LDA yourself, and should use a
3rd-party library like gensim or mallet (or any other reputable library you find
for LDA topic modeling) to do the topic modeling and may use libraries like
PyLDAvis to help present your results. You may also use jsLDA for both topic
modeling and generating some interesting visualizations (but you may not use
the example corpus presented there).
You should present your topics (or a selection of the topics you think are most
interesting/useful) in a table where the first column contains your own manually
3
assigned label for the topic (e.g., ”school”), and the subsequent columns contain
a the top 10 (or more) terms from that topic, sorted by their probability of
belonging to that topic, along with their probabilities (e.g. ”homework” (0.02),
”class” (0.01), and so on).
Finally, for each category, find the average distribution of all topics for documents
in that category and report the top 3-5 topics for each category. You
can determine this by taking the topic distribution for each document in a given
category and averaging the probabilities for each topic across all documents in
the category.

In [19]:
print(bow.shape)

(5324, 9210)


In [45]:


combined_docs = agr_docs + as_docs

text_documents = [' '.join(doc) for doc in combined_docs]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text_documents)
num_topics = 5
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=22)
lda_model.fit(X)



LatentDirichletAllocation(n_components=5, random_state=22)

In [46]:
def display_topics(model, vectorizer, no_top_words):
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        topic /= topic.sum()
        top_indices = topic.argsort()[:-no_top_words - 1:-1]
        top_terms = [feature_names[i] for i in top_indices]
        top_probabilities = [topic[i] for i in top_indices]
        for term, prob in zip(top_terms, top_probabilities):
            if not term.isdigit():
                print(f"    {term} ({prob:.4f})")

no_top_words = 10
print("Topics and top words with probabilities:\n")
display_topics(lda_model, vectorizer, no_top_words)

Topics and top words with probabilities:

Topic 0:
    agricultur (0.0138)
    food (0.0123)
    farmer (0.0122)
    usda (0.0116)
    farm (0.0104)
    program (0.0092)
    product (0.0092)
    produc (0.0086)
    increas (0.0074)
    research (0.0070)
Topic 1:
    forest (0.0193)
    servic (0.0100)
    program (0.0098)
    manag (0.0093)
    land (0.0093)
    state (0.0091)
    need (0.0087)
    author (0.0083)
    feder (0.0080)
    work (0.0075)
Topic 2:
    think (0.0151)
    go (0.0096)
    china (0.0091)
    defens (0.0088)
    would (0.0086)
    need (0.0085)
    know (0.0082)
    mr (0.0077)
    thing (0.0073)
    year (0.0069)
Topic 3:
    mr (0.0220)
    thank (0.0134)
    think (0.0118)
    work (0.0101)
    go (0.0100)
    would (0.0100)
    farm (0.0087)
    member (0.0086)
    make (0.0084)
    committe (0.0084)
Topic 4:
    market (0.0224)
    would (0.0120)
    product (0.0115)
    propos (0.0110)
    risk (0.0107)
    grower (0.0087)
    contract (0.0076)
    requir 